# 05: Advanced Multi-Agent Orchestration

This notebook explores the advanced multi-agent orchestration capabilities of the Agentic Framework. It demonstrates two distinct modes for orchestrating agent workflows: a static, plan-based approach (Router Mode with "Plan and Review") and a dynamic, graph-based system (Graph Mode with Loop Prevention).

## Getting Started

Before running this notebook, ensure that the Docker services for Chapter 06 are up and running. You can start them by navigating to the `chapter-06-advanced-multi-agent-orchestration` directory and executing the lifecycle script:

```bash
./start-chapter-resources.sh
```

Also, ensure you have an SSH session with port forwarding for OpenWebUI (`8902`), the Agent Gateway (`8083`), and the main MCP server (`8080`):

```bash
ssh -L 8902:localhost:8902 -L 8083:localhost:8083 -L 8080:localhost:8080 user@remote-server
```

## 1. Router Mode - Collaborative "Plan and Review"

This scenario demonstrates the new, more robust "Plan and Review" workflow in 'router' mode. In this mode, the agent team will collaboratively review and refine a plan before it is presented to the user for approval.

### Step 1: Create a Multi-Agent Session

We will create a multi-agent session in 'router' mode with specific roles for the worker agents.


In [ ]:
import openai
import json
import uuid

# Configuration
AGENT_GATEWAY_URL = "http://localhost:8083/v1" # Agent Gateway for chat interactions
MODEL_ID = "agentic-framework/scientific-agent-v1"
API_KEY = "not-a-real-key" # Dummy API key if auth is disabled

client = openai.OpenAI(
    base_url=AGENT_GATEWAY_URL,
    api_key=API_KEY,
)

# Generate a unique MCP session ID for this notebook run
mcp_session_id = str(uuid.uuid4())
print(f"Using MCP Session ID: {mcp_session_id}")

# Define the roles for the multi-agent team
roles = ["Bioinformatician", "Chemist", "Software Engineer"]

user_query = (
    f"Create a multi-agent session in 'router' mode. "
    f"The team should include a '{roles[0]}', a '{roles[1]}', and a '{roles[2]}'. "
    f"The overall goal is to perform a competitive analysis for a new drug candidate targeting protein P01112. "
    f"Use mcp_session_id: {mcp_session_id}"
)

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."}
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("
--- Agent Response ---")
    if response.choices:
        agent_response = response.choices[0].message.content
        print(agent_response)
        # Extract multi_agent_session_id from the response for subsequent steps
        # This is a simple regex, a more robust parser might be needed for real applications
        import re
        match = re.search(r'multi_agent_session_id: (multi-agent-[0-9a-f-]+)', agent_response)
        if match:
            multi_agent_session_id = match.group(1)
            print(f"Extracted multi_agent_session_id: {multi_agent_session_id}")
        else:
            print("Could not extract multi_agent_session_id from agent response.")
            multi_agent_session_id = None # Set to None if not found
    else:
        print("The agent did not return any choices.")
        multi_agent_session_id = None
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


### Step 2: Generate and Review Plan

Now, we will ask the agent to generate a detailed, multi-phase plan using the "Plan and Review" process. The agent team will review and refine the plan before presenting it to us.


In [ ]:
if multi_agent_session_id is None:
    print("Cannot proceed: multi_agent_session_id was not extracted from the previous step.")
else:
    task_description = (
        "Using the 'Plan and Review' process, generate a detailed, multi-phase plan for the session I just created, "
        "focusing on analyzing P01112's sequence, finding existing patents, and identifying similar compounds. "
        "Ensure the plan uses appropriate MCP tools and includes Python code execution steps for plotting where relevant. "
        "For Python plotting, remember to explicitly create figure objects (e.g., `fig, ax = plt.subplots()`) and avoid `plt.show()`. "
        "The figure object should be the last expression in the code block for capture. "
        "Also, ensure any Python code for plotting includes in-script package installations (e.g., using `subprocess`)."
    )
    
    user_query = (
        f"For multi_agent_session_id: {multi_agent_session_id}, "
        f"generate and review a plan for the following task: {task_description} "
        f"Use mcp_session_id: {mcp_session_id}"
)
    
    print(f"Sending query to agent: {user_query}")
    
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."}
                {"role": "user", "content": user_query}
            ],
            stream=False,
        )
        
        print("
--- Agent Response (Plan) ---")
        if response.choices:
            agent_response = response.choices[0].message.content
            print(agent_response)
        else:
            print("The agent did not return any choices.")
            
    except openai.APIConnectionError as e:
        print(f"Failed to connect to the Agent Gateway: {e}")
    except openai.APIStatusError as e:
        print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


### Step 3: Approve and Execute Plan

After reviewing the plan, we will approve it and instruct the agent to execute it. The agent will then delegate each step of the plan to the appropriate worker.


In [ ]:
if multi_agent_session_id is None:
    print("Cannot proceed: multi_agent_session_id is not set.")
else:
    user_query = (
        f"For multi_agent_session_id: {multi_agent_session_id}, "
        f"The plan looks good. Approve and execute it. "
        f"Use mcp_session_id: {mcp_session_id}"
)
    
    print(f"Sending query to agent: {user_query}")
    
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."}
                {"role": "user", "content": user_query}
            ],
            stream=False,
        )
        
        print("
--- Agent Response (Execution Report) ---")
        if response.choices:
            print(response.choices[0].message.content)
        else:
            print("The agent did not return any choices.")
            
    except openai.APIConnectionError as e:
        print(f"Failed to connect to the Agent Gateway: {e}")
    except openai.APIStatusError as e:
        print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


## 2. Graph Mode - Dynamic Problem Solving with Loop Prevention

This scenario demonstrates the dynamic 'graph' mode, now with enhanced resilience against infinite loops. In this mode, the supervisor agent will dynamically route tasks between worker agents based on the evolving state of the problem.

### Step 1: Create a Multi-Agent Session in Graph Mode

We will create a multi-agent session in 'graph' mode with specific roles.


In [ ]:
import openai
import json
import uuid

# Generate a unique MCP session ID for this notebook run
mcp_session_id_graph = str(uuid.uuid4())
print(f"Using MCP Session ID for Graph Mode: {mcp_session_id_graph}")

# Define the roles for the multi-agent team
roles_graph = ["Biologist", "Chemist"]

user_query = (
    f"Create a multi-agent session in 'graph' mode. "
    f"The team should include a '{roles_graph[0]}' and a '{roles_graph[1]}'. "
    f"Use mcp_session_id: {mcp_session_id_graph}"
)

print(f"Sending query to agent: {user_query}")

try:
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."}
            {"role": "user", "content": user_query}
        ],
        stream=False,
    )
    
    print("
--- Agent Response ---")
    if response.choices:
        agent_response = response.choices[0].message.content
        print(agent_response)
        import re
        match = re.search(r'multi_agent_session_id: (multi-agent-[0-9a-f-]+)', agent_response)
        if match:
            multi_agent_session_id_graph = match.group(1)
            print(f"Extracted multi_agent_session_id for Graph Mode: {multi_agent_session_id_graph}")
        else:
            print("Could not extract multi_agent_session_id for Graph Mode from agent response.")
            multi_agent_session_id_graph = None
    else:
        print("The agent did not return any choices.")
        multi_agent_session_id_graph = None
            
except openai.APIConnectionError as e:
    print(f"Failed to connect to the Agent Gateway: {e}")
except openai.APIStatusError as e:
    print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


### Step 2: Execute Dynamic Task

Now, we will execute a dynamic task using the graph-based session. The supervisor agent will dynamically route tasks between the worker agents.


In [ ]:
if multi_agent_session_id_graph is None:
    print("Cannot proceed: multi_agent_session_id_graph is not set.")
else:
    task_description_graph = (
        "Find the human KRAS protein sequence from UniProt, then perform a BLAST search against SwissProt, "
        "and finally identify chemical compounds in PubChem known to inhibit it. "
        "Include a plot of the BLAST results if possible, remembering the plotting guidelines for Python code."
    )
    
    user_query = (
        f"For multi_agent_session_id: {multi_agent_session_id_graph}, "
        f"execute the following task in graph mode: {task_description_graph} "
        f"Use mcp_session_id: {mcp_session_id_graph}"
)
    
    print(f"Sending query to agent: {user_query}")
    
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."}
                {"role": "user", "content": user_query}
            ],
            stream=False,
        )
        
        print("
--- Agent Response (Execution Report) ---")
        if response.choices:
            print(response.choices[0].message.content)
        else:
            print("The agent did not return any choices.")
            
    except openai.APIConnectionError as e:
        print(f"Failed to connect to the Agent Gateway: {e}")
    except openai.APIStatusError as e:
        print(f"The Agent Gateway returned an error status code: {e.status_code}. Response: {e.response.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


## 3. Terminate Multi-Agent Sessions

It's good practice to terminate multi-agent sessions when they are no longer needed to clean up resources.


In [ ]:
if multi_agent_session_id is not None:
    user_query_terminate_router = (
        f"Terminate the multi-agent session with ID: {multi_agent_session_id}. "
        f"Use mcp_session_id: {mcp_session_id}"
    )
    print(f"Sending query to agent: {user_query_terminate_router}")
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."}
                {"role": "user", "content": user_query_terminate_router}
            ],
            stream=False,
        )
        print("
--- Agent Response (Terminate Router Session) ---")
        if response.choices:
            print(response.choices[0].message.content)
    except Exception as e:
        print(f"Error terminating router session: {e}")

if multi_agent_session_id_graph is not None:
    user_query_terminate_graph = (
        f"Terminate the multi-agent session with ID: {multi_agent_session_id_graph}. "
        f"Use mcp_session_id: {mcp_session_id_graph}"
    )
    print(f"Sending query to agent: {user_query_terminate_graph}")
    try:
        response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."}
                {"role": "user", "content": user_query_terminate_graph}
            ],
            stream=False,
        )
        print("
--- Agent Response (Terminate Graph Session) ---")
        if response.choices:
            print(response.choices[0].message.content)
    except Exception as e:
        print(f"Error terminating graph session: {e}")

print("
All multi-agent sessions terminated (if they were active).")


This notebook provided a comprehensive overview of the advanced multi-agent orchestration capabilities, including both router and graph modes. This concludes the tutorial for Chapter 06.
